# Data Engineering Capstone Project

Data Engineering Nanodegree conclusion project.

## Immigration in the US



In [44]:
import numpy as np
import pandas as pd
from datetime import datetime
import psycopg2

The following project consists in building a database model for immigration data in the United States of America. Analyses on this data can be useful for both government and business decision making.

#### Datasets

- I94 Immigration Data: This data comes from the US National Tourism and Trade Office, but for this notebook only a small sample will be used. [This is where the data comes from](https://travel.trade.gov/research/reports/i94/historical/2016.html).

- World Temperature Data: This dataset came from ---. [You can read more about it here](---).

- U.S. City Demographic Data: This data comes from OpenSoft. [You can read more about it here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).

- Airport Code Table: This is a simple table of airport codes and corresponding cities. [It comes from here](https://datahub.io/core/airport-codes#data).


## Exploring the Data




#### I94 Immigration

In [47]:
immigration = pd.read_csv('immigration_data_sample.csv')
immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


#### World temperature by city

In this case, a folder with JSON files contains temperature data from US cities.

In [60]:
##################
# https://openweathermap.org/history

#### us cities

In [61]:
us_cities = pd.read_csv('us-cities-demographics.csv', sep=';')
us_cities.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


#### airports

In [55]:
airports = pd.read_csv('airport-codes_csv.csv')
airports.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Data Quality Checks

The data quality checks include:
 * Integrity constraints on the relational database.
 * Source/Count checks to ensure completeness.
 * try/catch code blocks to double check on duplicates.

### Cleaning Steps

- I94 immigration:
-
- us cities: Nothing special to be changed, some columns could be INT instead of DOUBLE or FLOAT, but pandas does not permit that without taking some action on NA values(like removing them or filling with zeros), which is terrible. I'd rather model the database using FLOAT data type than restricting analysts options on handling missing data.

- airports: Here data is not in the first normal form, so lets change that by writing coordinates in separate columns. 

In [59]:
# Data Cleaning

## immigration

## world_temperature

## airports
airports['latitude'] = airports['coordinates'].apply(lambda x: float(x.split(',')[0]))
airports['longitude'] = airports['coordinates'].apply(lambda x: float(x.split(',')[1]))
airports = airports.drop(columns=['coordinates'])

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,latitude,longitude
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,-74.933601,40.070801
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,-101.473911,38.704022
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,-151.695999,59.949200
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,-86.770302,34.864799
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,-91.254898,35.608700


## Data Model

### Schema

Event though they have some limitations, relational databases are consolidated and well suited for Big Data situations. For many of the upcoming analysis in this notebook there is no need to use Cloud solutions such as AWS Redshift, so I opted to stick to the basics and at the end of this document you can find some scenarios(where reasonable solutions are adressed) that require more speed and scalability.

The schema is described as follows:

#### ETL pipeline


Extracting csv files content, checking for inconsistencies across datasets, and finally inserting data into tables described by the Schema.

In [ ]:
# create tables

#### ETL pipeline


Extracting csv files content, checking for inconsistencies across datasets, and finally inserting data into tables described by the Schema.

In [1]:
# insert statements

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [7]:
# insert records into tables

### Data analysis

Some analysis on immigration data could be:

- Find patterns of gender and/or age(differences in visa type or airline chosen);
- Rank airlines and routes by number of immigrants.
- Track flow of passengers flying to great urban centers.
- Find patterns of immigration based on seasons of the year.
- Check if exists some relation between a city thermal amplitude and emigration.

In [ ]:
# Data analysis

## Complete Project Write Up

### WORK IN PROGRESS

As mentioned earlier, using a relational database may be enough for most applications. Moving to the cloud is always an option and some situations where this is reasonable are described below.

Possible decisions for alternate scenarios:

- "Data increases by 100x": Instead of using a structured database in disk, a Data Lake could be used, as the project especifies that various types and sources of data(structured or unstructured) can be explored. It is not possible to tell upfront which will be useful. A possible approach is to launch an EMR Cluster and designing the schema on read.

- "The data populates a dashboard that must be updated on a daily basis by 7am every day": Schedule tasks using Apache Airflow.

- "The database needed to be accessed by 100+ people": Redshift Clusters can handle the traffic without changing the RDBMS, but it's important to monitor AWS billing to avoid unnecessary costs.